In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=50fe1cbb00812ec02b14797b18f77728dce9837ba9c524d83146d83a65a4cf26
  Stored in directory: /Users/nicholasmajeski/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [9]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [27]:
koi_disposition = df['koi_disposition']
koi_disposition_copy = koi_disposition.copy()

koi_disposition_encoded = pd.get_dummies(koi_disposition_copy)
koi_disposition_encoded.head()

,CANDIDATE,CONFIRMED,FALSE POSITIVE
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, koi_disposition_encoded, random_state=42)
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth
6122,6.768901,133.077240,0.150,3.61600,123.1
6370,0.733726,132.020050,0.291,2.30900,114.6
2879,7.652707,134.460380,0.970,79.89690,641.1
107,7.953547,174.662240,0.300,2.63120,875.4
29,4.959319,172.258529,0.831,2.22739,9802.0


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [52]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Train the Model



In [59]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf = rf.fit(X_train_scaled, y_train_scaled)

print(f"Training Data Score: {rf.score(X_train_scaled, y_train_scaled)}")
print(f"Testing Data Score: {rf.score(X_test_scaled, y_test_scaled)}")

Training Data Score: 0.9597558649628075
Testing Data Score: 0.5617848970251716


/Users/nicholasmajeski/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [62]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [1, 5, 10, 50, 100, 200, 400, 800]}
grid = GridSearchCV(rf, param_grid, verbose=3, cv=5)

In [63]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train_scaled)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] n_estimators=1 ..................................................
[CV] ...................... n_estimators=1, score=0.593, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ...................... n_estimators=1, score=0.617, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ...................... n_estimators=1, score=0.630, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ...................... n_estimators=1, score=0.611, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ...................... n_estimators=1, score=0.602, total=   0.0s
[CV] n_estimators=5 ..................................................
[CV] ...................... n_estimators=5, score=0.594, total=   0.0s
[CV] n_estimators=5 ..................................................
[CV] ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ...................... n_estimators=5, score=0.612, total=   0.0s
[CV] n_estimators=5 ..................................................
[CV] ...................... n_estimators=5, score=0.619, total=   0.1s
[CV] n_estimators=5 ..................................................
[CV] ...................... n_estimators=5, score=0.642, total=   0.0s
[CV] n_estimators=10 .................................................
[CV] ..................... n_estimators=10, score=0.570, total=   0.1s
[CV] n_estimators=10 .................................................
[CV] ..................... n_estimators=10, score=0.602, total=   0.1s
[CV] n_estimators=10 .................................................
[CV] ..................... n_estimators=10, score=0.581, total=   0.1s
[CV] n_estimators=10 .................................................
[CV] ..................... n_estimators=10, score=0.553, total=   0.1s
[CV] n_estimators=10 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=10, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [64]:
print(grid.best_params_)
print(grid.best_score_)

{'n_estimators': 800}
0.6213999618539005


# Save the Model

In [65]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'nicholas_majeski.sav'
joblib.dump(rf, filename)

['nicholas_majeski.sav']